In [1]:
setwd(paste0(Sys.getenv('ROOT'), '/R'))
getwd()

[1] "/mnt/hdd/develop/kaggle/house-prices/R/R"

In [16]:
source('main.R')

df.train <- data_frame(
    BldgType       = c('A', 'A', 'A', 'B', 'B', 'C', 'C'),
    Quality        = c( 1,   1,   2,   1,   3,   3,   2),
    price          = c(50,   40,  30,  30,   20,   10,   20)
) %>% mutate(id = 1:nrow(.))

df.test <- data_frame(
    BldgType       = c('A', 'A', 'B', 'B', 'C', 'D'),
    Quality        = c( 2,   1,   2,   2,   4,   3),
) %>% mutate(id = 1:nrow(.))

df.train
df.test

BldgType,Quality,price,id
A,1,50,1
A,1,40,2
A,2,30,3
B,1,30,4
B,3,20,5
C,3,10,6
C,2,20,7


BldgType,Quality,id
A,2,1
A,1,2
B,2,3
B,2,4
C,4,5
D,3,6


In [56]:
df.train.tran <-
    df.train %>%
    gather(var.name, var.value, -price, -id) %>%
    group_by(var.name, var.value) %>%
    mutate(avg.price = mean(price)) %>%
    ungroup %>% 
    select(-var.value, -price) %>% 
    spread(var.name, avg.price) %>% 
    arrange(id)

df.train.tran

Z <- 
    df.train.tran %>% 
    inner_join(df.train %>% rename(old.BldgType=BldgType, old.Quality=Quality), by='id')

id,BldgType,Quality
1,40,40
2,40,40
3,40,25
4,25,40
5,25,15
6,15,15
7,15,25


In [62]:
Z %>%
    group_by(old.BldgType) %>%
    mutate(avg.BldgType = mean(price)) %>%
    group_by(old.Quality) %>%
    mutate(avg.Quality = mean(price)) %>%
    mutate(
        avg.BldgType == BldgType,
         avg.Quality == Quality
    )

id,BldgType,Quality,old.BldgType,old.Quality,price,avg.BldgType,avg.Quality,avg.BldgType == BldgType,avg.Quality == Quality
1,40,40,A,1,50,40,40,TRUE,TRUE
2,40,40,A,1,40,40,40,TRUE,TRUE
3,40,25,A,2,30,40,25,TRUE,TRUE
4,25,40,B,1,30,25,40,TRUE,TRUE
5,25,15,B,3,20,25,15,TRUE,TRUE
6,15,15,C,3,10,15,15,TRUE,TRUE
7,15,25,C,2,20,15,25,TRUE,TRUE


In [83]:
train.log <- 
    df.train %>%
    gather(var.name, var.value, -price, -id) %>%
    mutate(src = 'train')

test.long <-
    df.test %>%
        gather(var.name, var.value, -id) %>%
        mutate(src = 'test')

long <- train.log %>% bind_rows(test.long)

long %>% head

price,id,var.name,var.value,src
50,1,BldgType,A,train
40,2,BldgType,A,train
30,3,BldgType,A,train
30,4,BldgType,B,train
20,5,BldgType,B,train
10,6,BldgType,C,train


In [87]:
long %>%
    group_by(var.name, var.value) %>%
    arrange(var.name, var.value) %>%
    mutate(avg.price = mean(price, na.rm=T))

price,id,var.name,var.value,src,avg.price
50,1,BldgType,A,train,40
40,2,BldgType,A,train,40
30,3,BldgType,A,train,40
NA,1,BldgType,A,test,40
NA,2,BldgType,A,test,40
30,4,BldgType,B,train,25
20,5,BldgType,B,train,25
NA,3,BldgType,B,test,25
NA,4,BldgType,B,test,25
10,6,BldgType,C,train,15
